# Recurrent Neural Networks
You should build an end-to-end machine learning pipeline using a recurrent neural network model. In particular, you should do the following:
- Load the `jena climate` dataset using [Pandas](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html). You can find this dataset in the [keras repository](https://keras.io/examples/timeseries/timeseries_weather_forecasting/).
- Split the dataset into training, validation, and test sets. Note that you cannot split time series using [Scikit-Learn](https://keras.io/examples/timeseries/timeseries_weather_forecasting/).
- Build an end-to-end machine learning pipeline, including a [recurrent neural network](https://keras.io/examples/timeseries/timeseries_weather_forecasting/) model.
- Optimize your pipeline by validating your design decisions.
- Test the best pipeline on the test set and report various [evaluation metrics](https://scikit-learn.org/0.15/modules/model_evaluation.html).  
- Check the documentation to identify the most important hyperparameters, attributes, and methods of the model. Use them in practice.

In [1]:
"""
1. Load the data
2. Take a sample from the dataset select measurements are recorded at 12pm.
3. Split the data into training, validation and test splits(70% for tarin, 15% for validation, 15% for test)
4. Create Sequences
5. Design and Traun your model
6. Test your model
"""

'\n1. Load the data\n2. Take a sample from the dataset select measurements are recorded at 12pm.\n3. Split the data into training, validation and test splits(70% for tarin, 15% for validation, 15% for test)\n4. Create Sequences \n5. Design and Traun your model\n6. Test your model \n'

In [2]:
!pip install tensorflow

In [3]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split


### EDA

In [4]:
df = pd.read_csv('https://storage.googleapis.com/tensorflow/tf-keras-datasets/jena_climate_2009_2016.csv.zip')
df.head()

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


### Data Preprocessing

In [5]:
df['Date Time'] = pd.to_datetime(df['Date Time'], format='%d.%m.%Y %H:%M:%S')

df_10_AM = df[df['Date Time'].dt.hour == 10].set_index('Date Time')
df_10_AM.head()

,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
Date Time,,,,,,,,,,,,,,
2009-01-01 10:00:00,1000.27,-7.04,266.10,-8.17,91.6,3.60,3.30,0.30,2.05,3.29,1307.76,1.45,3.00,292.6
2009-01-01 10:10:00,1000.43,-7.03,266.10,-8.16,91.6,3.60,3.30,0.30,2.05,3.30,1307.91,1.12,2.75,266.3
2009-01-01 10:20:00,1000.54,-7.15,265.97,-8.34,91.1,3.57,3.25,0.32,2.02,3.25,1308.67,2.12,3.25,267.2
2009-01-01 10:30:00,1000.68,-7.26,265.85,-8.47,91.0,3.54,3.22,0.32,2.00,3.22,1309.41,1.54,3.25,258.8
2009-01-01 10:40:00,1000.78,-7.34,265.76,-8.57,90.8,3.51,3.19,0.32,1.99,3.19,1309.96,1.81,3.00,257.0


### Splitting the data into train

In [6]:
train_percent = 0.8
validation_percent = 0.1
test_percent = 0.1

total_rows = len(df_10_AM)
train_rows = int(total_rows * train_percent)
validation_rows = int(total_rows * validation_percent)
test_rows = int(total_rows * test_percent)

train_data = df_10_AM[:train_rows]
validation_data = df_10_AM[train_rows:train_rows + validation_rows]
test_data = df_10_AM[train_rows + validation_rows:]

print("Shape of the 10 am sampled dataset:", df_10_AM.shape)
print("Shape of the training set:", train_data.shape)
print("Shape of the validation set:", validation_data.shape)
print("Shape of the test set:", test_data.shape)

Shape of the 10 am sampled dataset: (17523, 14)
Shape of the training set: (14018, 14)
Shape of the validation set: (1752, 14)
Shape of the test set: (1753, 14)


### Creating Sequences

In [7]:
target_column = 'T (degC)'
sequence_length = 24
prediction_horizon = 1

train_data_np = train_data.values
validation_data_np = validation_data.values
test_data_np = test_data.values

target_column_index = list(train_data.columns).index(target_column)

train_sequence_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=train_data_np,
    targets=train_data_np[:, target_column_index][sequence_length + prediction_horizon - 1:], # Target is the value at the prediction horizon
    sequence_length=sequence_length,
    sampling_rate=1,
    start_index=0,
    end_index=None,
    shuffle=False,
    batch_size=32
)

validation_sequence_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=validation_data_np,
    targets=validation_data_np[:, target_column_index][sequence_length + prediction_horizon - 1:],
    sequence_length=sequence_length,
    sampling_rate=1,
    start_index=0,
    end_index=None,
    shuffle=False,
    batch_size=32
)

test_sequence_dataset = tf.keras.preprocessing.timeseries_dataset_from_array(
    data=test_data_np,
    targets=test_data_np[:, target_column_index][sequence_length + prediction_horizon - 1:],
    sequence_length=sequence_length,
    sampling_rate=1,
    start_index=0,
    end_index=None,
    shuffle=False,
    batch_size=32
)

# Printing the shapes of the first batch of each dataset
print("\nDataset Batch Shapes:")
for batch in train_sequence_dataset.take(1):
    inputs, targets = batch
    print("Train Input batch shape:", inputs.numpy().shape)
    print("Train Target batch shape:", targets.numpy().shape)

for batch in validation_sequence_dataset.take(1):
    inputs, targets = batch
    print("Validation Input batch shape:", inputs.numpy().shape)
    print("Validation Target batch shape:", targets.numpy().shape)

for batch in test_sequence_dataset.take(1):
    inputs, targets = batch
    print("Test Input batch shape:", inputs.numpy().shape)
    print("Test Target batch shape:", targets.numpy().shape)



Dataset Batch Shapes:
Train Input batch shape: (32, 24, 14)
Train Target batch shape: (32,)
Validation Input batch shape: (32, 24, 14)
Validation Target batch shape: (32,)
Test Input batch shape: (32, 24, 14)
Test Target batch shape: (32,)


### Training Model

In [10]:
for batch in train_sequence_dataset.take(1):
    inputs_batch, targets_batch = batch
    input_shape = (inputs_batch.shape[1], inputs_batch.shape[2])
    break

model = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=input_shape),
        tf.keras.layers.LSTM(units = 64, return_sequences=True),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.LSTM(units = 32, return_sequences=True),
        tf.keras.layers.Dense(units = 512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.Dense(units = 256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.Dense(units = 128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.Dense(units = 64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        tf.keras.layers.Dense(units = 10, activation='softmax'),
        tf.keras.layers.Dense(units = prediction_horizon)
    ]
)

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 24, 64)         │        20,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 24, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 24, 32)         │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 24, 512)        │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 24, 256)        │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 24, 128)        │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 24, 64)         │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 24, 10)         │           650 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 24, 1)          │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 222,677 (869.83 KB)

 Trainable params: 222,677 (869.83 KB)

 Non-trainable params: 0 (0.00 B)

### Saving checkpoint and execurting model

In [11]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse", metrics=['accuracy'])

path_checkpoint = "model_checkpoint.weights.h5"
es_callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", min_delta=0, patience=10, restore_best_weights=True)

modelckpt_callback = tf.keras.callbacks.ModelCheckpoint(
    monitor="val_loss",
    filepath=path_checkpoint,
    verbose=1,
    save_weights_only=True,
    save_best_only=True,
)

history = model.fit(
    train_sequence_dataset,
    epochs=500,
    validation_data=validation_sequence_dataset,
    callbacks=[es_callback, modelckpt_callback],
    shuffle=False,
    verbose=1
)

Epoch 1/500
437/438 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 3.1750e-06 - loss: 157.9472
Epoch 1: val_loss improved from inf to 176.96149, saving model to model_checkpoint.weights.h5
438/438 ━━━━━━━━━━━━━━━━━━━━ 45s 87ms/step - accuracy: 3.4861e-06 - loss: 157.9496 - val_accuracy: 0.0000e+00 - val_loss: 176.9615
Epoch 2/500
187/438 ━━━━━━━━━━━━━━━━━━━━ 20s 81ms/step - accuracy: 4.9833e-04 - loss: 135.6025

KeyboardInterrupt: 

In [ ]:
def visualize_loss(history, title):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]
    epochs = range(len(loss))
    plt.figure()
    plt.plot(epochs, loss, "b", label="Training loss")
    plt.plot(epochs, val_loss, "r", label="Validation loss")
    plt.title(title)
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()
    plt.show()


visualize_loss(history, "Training and Validation Loss")